In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import xgboost

pd.set_option('future.no_silent_downcasting', True)

In [42]:
base = Path('/home/zach/datasets/ieee-fraud-detection')
train = pd.read_csv(base / 'train_transaction.csv')

# test = pd.read_csv(base / 'test_transaction.csv')
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
cols = train.columns.tolist()
print(cols)

['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V

- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

In [62]:
# TODO find multicollinearity between masked features


def prepare_feature_sets(X):

    X = X.copy()


    print(counting)
    print(match)
    print(timedelta)

    # cat encode M4
    X['M4'] = X['M4'].astype('category').cat.codes

    X.loc[:, match] = (
        X.loc[:, match]
            .apply(lambda col: col.map({'F': 0, 'T': 1}))
            .fillna(0)
            .astype('int8')
    )

    X[match] = X[match].astype('int8')

    return X

counting = [col for col in cols if col[0] == 'C']
match = [col for col in cols if col[0] == 'M']
timedelta = [col for col in cols if col[0] == 'D']

prepared = prepare_feature_sets(train)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']
['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']


In [65]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

for feature_set in [counting, match, timedelta]:
    print(f"VIF for feature set starting with {feature_set[0][0]}")
    X = prepared[feature_set].fillna(0)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    print(vif_data.sort_values(by="VIF", ascending=False).head(20))

VIF for feature set starting with C
   feature          VIF
11     C12  4195.722689
6       C7  2953.148577
10     C11  2313.900499
7       C8  1614.232379
3       C4   856.079141
5       C6   650.245064
9      C10   530.471117
0       C1   513.258502
13     C14   368.179637
1       C2   347.255671
8       C9    69.359449
12     C13    41.240021
4       C5    15.059040
2       C3     1.000764
VIF for feature set starting with M


/home/zach/repos/venvs/fraud/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


  feature       VIF
2      M3  0.002791
1      M2  0.001715
8      M9  0.001576
4      M5  0.000539
5      M6  0.000340
7      M8 -0.001298
0      M1 -0.002387
6      M7 -0.005716
3      M4       NaN
VIF for feature set starting with D
   feature        VIF
0       D1  27.487548
1       D2  25.606578
14     D15   4.778544
9      D10   3.074498
3       D4   2.938661
5       D6   2.922974
11     D12   2.390084
10     D11   2.222142
6       D7   1.801675
4       D5   1.733324
2       D3   1.497494
8       D9   1.389578
7       D8   1.372984
12     D13   1.223705
13     D14   1.086763


High multicollinearity between features D1 and D2 suggest they should not be included in the model.
Extremely high multicollinearity between numerous count columns (C12, C7, C11, C8, C4, C6, C10, C1, C14, C2, C9, C13, C5) suggest they should not be included, either.